In [1]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T

from sklearn.metrics import roc_auc_score

In [2]:
cuda = torch.cuda.is_available()

In [3]:
cuda

True

In [4]:
transform = T.Compose([T.Resize(224), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

In [5]:
train_dataset = torchvision.datasets.ImageFolder(root='classification_data/train_data/', 
                                                       transform=transform)
val_dataset = torchvision.datasets.ImageFolder(root='classification_data/val_data/', 
                                                       transform=transform)
test_dataset = torchvision.datasets.ImageFolder(root='classification_data/test_data/', 
                                                       transform=transform)

In [6]:
#Hyperparameters
batch_size = 128
num_workers = 8 if cuda else 0
pin_memory = True if cuda else False
numEpochs = 30
#num_feats = 3
lr = 0.001 #0.001
patience = 5
factor = 0.316
#learningRate = 1e-2
weight_decay = 5e-5

num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
torch.min(val_dataset[0][0])

tensor(0.)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)

In [8]:
model = torchvision.models.resnet18(num_classes = 4000)

In [9]:
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

In [10]:
def train(model, data_loader, test_loader, ver_loader, task='Classification'):
    model.train()
    if task == 'Classification':
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=patience, factor=factor, verbose=True)
    else:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=patience, factor=factor, verbose=True, mode='max')
    for epoch in tqdm(range(numEpochs)):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        val_loss, val_acc = test_classify(model, test_loader)
        #train_loss, train_acc = test_classify(model, data_loader)
        print('Val Loss: {:.4f}\tVal Accuracy: {:.4f}'.format(val_loss, val_acc))
        
        if task == 'Classification':
            
            scheduler.step(val_loss)
        else:
            roc_score = test_verify(model, ver_loader)
            print('Roc score: {:.4f}'.
                  format(roc_score))
            scheduler.step(roc_score)


def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total


def test_verify(model, test_loader):
    with torch.no_grad():
        newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
        newmodel.to(device)
        newmodel.eval()
        similarities = []
        true_labels = []
        for batch_num, (feats1, feats2, labels) in enumerate(test_loader):
            feats1, feats2 = feats1.to(device), feats2.to(device)
            #feats1 = feats1.to(device)
            output1 = newmodel(feats1)
            output2 = newmodel(feats2)
        
            cos = nn.CosineSimilarity()
        
            sim = cos(output1, output2)
            similarities.extend(sim)
            true_labels.extend(labels)
        
            del feats1
            del feats2
            del labels
    
        true_labels = np.array(true_labels)
        similarities = np.array(similarities)
        model.train()
        return roc_auc_score(true_labels, similarities)

In [11]:
model.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)

In [12]:
class VerificationDataset(Dataset):
    def __init__(self, filename):
        self.file1_list = []
        self.file2_list = []
        self.target_list = []
        infile = open(filename , "r" )
        
        for line in infile :
            imfile1, imfile2, match = line.split()
            self.file1_list.append(imfile1)
            self.file2_list.append(imfile2)
            self.target_list.append(int(match))
        
        infile.close()
        

    def __len__(self):
        return len(self.file1_list)

    def __getitem__(self, index):
        img1 = Image.open(self.file1_list[index])
        img1 = transform(img1)
        img2 = Image.open(self.file2_list[index])
        img2 = transform(img2)
        label = self.target_list[index]
        return img1, img2, label

In [13]:
verification_set = VerificationDataset("verification_pairs_val.txt")

In [14]:
ver_dataloader = DataLoader(verification_set, batch_size=32, shuffle=False, num_workers=4, pin_memory=False)

In [15]:
model.to(device)
train(model, train_dataloader, val_dataloader, ver_dataloader, task='Verification')

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1	Batch: 50	Avg-Loss: 8.4857
Epoch: 1	Batch: 100	Avg-Loss: 8.2939
Epoch: 1	Batch: 150	Avg-Loss: 8.2876
Epoch: 1	Batch: 200	Avg-Loss: 8.2799
Epoch: 1	Batch: 250	Avg-Loss: 8.2676
Epoch: 1	Batch: 300	Avg-Loss: 8.2433
Epoch: 1	Batch: 350	Avg-Loss: 8.2279
Epoch: 1	Batch: 400	Avg-Loss: 8.1759
Epoch: 1	Batch: 450	Avg-Loss: 8.1564
Epoch: 1	Batch: 500	Avg-Loss: 8.1001
Epoch: 1	Batch: 550	Avg-Loss: 8.0678
Epoch: 1	Batch: 600	Avg-Loss: 8.0486
Epoch: 1	Batch: 650	Avg-Loss: 8.0037
Epoch: 1	Batch: 700	Avg-Loss: 7.9629
Epoch: 1	Batch: 750	Avg-Loss: 7.9062
Epoch: 1	Batch: 800	Avg-Loss: 7.8615
Epoch: 1	Batch: 850	Avg-Loss: 7.8277
Epoch: 1	Batch: 900	Avg-Loss: 7.7608
Epoch: 1	Batch: 950	Avg-Loss: 7.6957
Epoch: 1	Batch: 1000	Avg-Loss: 7.6570
Epoch: 1	Batch: 1050	Avg-Loss: 7.6016
Epoch: 1	Batch: 1100	Avg-Loss: 7.5141
Epoch: 1	Batch: 1150	Avg-Loss: 7.4476
Epoch: 1	Batch: 1200	Avg-Loss: 7.4009
Epoch: 1	Batch: 1250	Avg-Loss: 7.3565
Epoch: 1	Batch: 1300	Avg-Loss: 7.2809
Epoch: 1	Batch: 1350	Avg-Loss: 7

  3%|▎         | 1/30 [21:10<10:13:55, 1270.20s/it]

Roc score: 0.8573
Epoch: 2	Batch: 50	Avg-Loss: 4.9236
Epoch: 2	Batch: 100	Avg-Loss: 4.9101
Epoch: 2	Batch: 150	Avg-Loss: 4.8602
Epoch: 2	Batch: 200	Avg-Loss: 4.8219
Epoch: 2	Batch: 250	Avg-Loss: 4.7363
Epoch: 2	Batch: 300	Avg-Loss: 4.7469
Epoch: 2	Batch: 350	Avg-Loss: 4.7217
Epoch: 2	Batch: 400	Avg-Loss: 4.6423
Epoch: 2	Batch: 450	Avg-Loss: 4.6095
Epoch: 2	Batch: 500	Avg-Loss: 4.5714
Epoch: 2	Batch: 550	Avg-Loss: 4.5109
Epoch: 2	Batch: 600	Avg-Loss: 4.4899
Epoch: 2	Batch: 650	Avg-Loss: 4.4346
Epoch: 2	Batch: 700	Avg-Loss: 4.4285
Epoch: 2	Batch: 750	Avg-Loss: 4.3358
Epoch: 2	Batch: 800	Avg-Loss: 4.3098
Epoch: 2	Batch: 850	Avg-Loss: 4.2476
Epoch: 2	Batch: 900	Avg-Loss: 4.2880
Epoch: 2	Batch: 1200	Avg-Loss: 4.0633
Epoch: 2	Batch: 1250	Avg-Loss: 4.1394
Epoch: 2	Batch: 1300	Avg-Loss: 4.0352
Epoch: 2	Batch: 1350	Avg-Loss: 4.0144
Epoch: 2	Batch: 1400	Avg-Loss: 4.0058
Epoch: 2	Batch: 1450	Avg-Loss: 3.9433
Epoch: 2	Batch: 1500	Avg-Loss: 3.9201
Epoch: 2	Batch: 1550	Avg-Loss: 3.8641
Epoch: 2	Batc

  7%|▋         | 2/30 [42:31<9:54:17, 1273.48s/it] 

...

 57%|█████▋    | 17/30 [5:59:25<4:35:06, 1269.70s/it]

Roc score: 0.9279
Epoch: 18	Batch: 50	Avg-Loss: 0.0875
Epoch: 18	Batch: 100	Avg-Loss: 0.0783
Epoch: 18	Batch: 150	Avg-Loss: 0.0684
Epoch: 18	Batch: 200	Avg-Loss: 0.0820
Epoch: 18	Batch: 250	Avg-Loss: 0.0750
Epoch: 18	Batch: 300	Avg-Loss: 0.0744
Epoch: 18	Batch: 350	Avg-Loss: 0.0780
Epoch: 18	Batch: 400	Avg-Loss: 0.0734
Epoch: 18	Batch: 450	Avg-Loss: 0.0750
Epoch: 18	Batch: 500	Avg-Loss: 0.0736
Epoch: 18	Batch: 550	Avg-Loss: 0.0749
Epoch: 18	Batch: 600	Avg-Loss: 0.0793
Epoch: 18	Batch: 650	Avg-Loss: 0.0806
Epoch: 18	Batch: 700	Avg-Loss: 0.0839
Epoch: 18	Batch: 750	Avg-Loss: 0.0775
Epoch: 18	Batch: 800	Avg-Loss: 0.0857
Epoch: 18	Batch: 850	Avg-Loss: 0.0843
Epoch: 18	Batch: 900	Avg-Loss: 0.0835
Epoch: 18	Batch: 950	Avg-Loss: 0.0853
Epoch: 18	Batch: 1000	Avg-Loss: 0.0916
Epoch: 18	Batch: 1050	Avg-Loss: 0.0927
Epoch: 18	Batch: 1100	Avg-Loss: 0.0965
Epoch: 18	Batch: 1150	Avg-Loss: 0.1056
Epoch: 18	Batch: 1200	Avg-Loss: 0.1023
Epoch: 18	Batch: 1250	Avg-Loss: 0.1081
Epoch: 18	Batch: 1300	Avg-L

 60%|██████    | 18/30 [6:20:31<4:13:42, 1268.56s/it]

Roc score: 0.9297


KeyboardInterrupt: 

In [ ]:
test_classify(model, val_dataloader)

In [ ]:
print(model)

In [16]:
torch.save(model.state_dict(), './18epochswd5e-5-adam-lranneal-resize.pt')

In [ ]:
model.load_state_dict(torch.load('./26epochswd5e-5-adam-lranneal.pt'))

In [ ]:
test_verify(model, ver_dataloader)

In [17]:
class VerificationTestDataset(Dataset):
    def __init__(self, filename):
        self.file1_list = []
        self.file2_list = []
        
        infile = open(filename , "r" )
        
        for line in infile :
            imfile1, imfile2 = line.split()
            self.file1_list.append(imfile1)
            self.file2_list.append(imfile2)
            
        
        infile.close()
        

    def __len__(self):
        return len(self.file1_list)

    def __getitem__(self, index):
        img1 = Image.open(self.file1_list[index])
        img1 = transform(img1)
        img2 = Image.open(self.file2_list[index])
        img2 = transform(img2)
        
        return img1, img2

In [18]:
ver_test_set = VerificationTestDataset("verification_pairs_test.txt")

In [19]:
vertest_dataloader = DataLoader(ver_test_set, batch_size=32, shuffle=False, num_workers=4, pin_memory=False)

In [20]:
def test_final(model, test_loader):
    with torch.no_grad():
        newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))
        newmodel.to(device)
        newmodel.eval()
        similarities = []
        
        for batch_num, (feats1, feats2) in enumerate(test_loader):
            feats1, feats2 = feats1.to(device), feats2.to(device)
            #feats1 = feats1.to(device)
            output1 = newmodel(feats1)
            output2 = newmodel(feats2)
        
            cos = nn.CosineSimilarity()
        
            sim = cos(output1, output2)
            sim = sim.cpu()
            similarities.extend(sim.numpy())
            
        
            del feats1
            del feats2
            
    
        
        similarities = np.array(similarities)
        
        return similarities

In [21]:
test_sims = test_final(model, vertest_dataloader)

In [22]:
test_sims = np.reshape(test_sims, (51835,))

In [ ]:
test_sims

In [23]:
import pandas as pd
data = {'Id':["{} {}".format(a, b) for a, b in zip(ver_test_set.file1_list, ver_test_set.file2_list)],
       'Category': test_sims}



In [24]:
df = pd.DataFrame(data)

In [ ]:
df

In [25]:
df.to_csv("submission.csv", header=True, index=False)